In [29]:
import copy

# Merge two dictionaries or dictionaries

In [41]:
global_dict = {
    "shared": {"data_paths": {"global_cache": "/nesi/project/niwa03440/cylc_test/"},
               "output": {"crs": {"horizontal": 2193, "vertical": 7839}},
               "apis": {"vector": {"linz": {"land": {"layers": [51559]}, "bathymetry_contours": {"layers": [50849]}}}},
               "general": {"bathymetry_contours_z_label": "valdco", "lidar_classifications_to_keep": [2, 9]}},
    "rivers": {"output": {"grid_params": {"resolution": 1}},
               "processing": {"chunk_size": 1400},
               "general": {"drop_offshore_lidar": True, "interpolate_missing_values": False},
               "rivers": {"veg_lidar_classifications_to_keep": [2, 3, 4, 5, 9],
                          "cross_section_spacing": 10,
                          "network_file_relative_path": "data/rec1_flow_and_friction.geojson",
                          "network_columns": {"id": "NZREACH", "to_node": "to_node", "from_node": "from_node", "flow": "flow", "mannings_n": "n", "area": "CATCHAREA"}}}, 
    "waterways": {"output": {"grid_params": {"resolution": 1}},
                  "processing": {"chunk_size": 2500}, 
                  "drains": {"widths": {"drain": 5, "stream": 7.5, "river": 10}}},
    "dem": {"output": {"grid_params": {"resolution": 8}},
            "processing": {"chunk_size": 300},
            "data_paths": {"river_polygons": ["river_polygon.geojson", "fan_polygon.geojson",
                                              "closed_waterways_polygon.geojson", "open_waterways_polygon.geojson"],
                           "river_bathymetry": ["river_bathymetry.geojson", "fan_bathymetry.geojson",
                                                "closed_waterways_elevation.geojson", "open_waterways_elevation.geojson"]},
            "general": {"drop_offshore_lidar": True, "bathymetry_points_type": ["rivers", "fans", "waterways", "waterways"], "interpolation_method": "linear",
                        "bathymetry_points_z_label": ["bed_elevation_Rupp_and_Smart", "bed_elevation_Rupp_and_Smart", "elevation", "elevation"]}},
    "roughness": {"output": {"grid_params": {"resolution": 1}},
                  "processing": {"chunk_size": 300},
                  "general": {"interpolation_method": "linear", "lidar_classifications_to_keep": [1, 2, 3, 4, 5, 9]}}
}
    

In [42]:
catchment_dict = {
    "shared": {"apis": {"lidar": {"open_topography": {"NZ20_Westport": True}}},
               "general": {"elevation_range": [-10, 4000]}},
    "rivers": {"rivers": { "osm_id": 132793862, "area_threshold": 100000000, "minimum_slope": 0.002, "network_id": 9003742,
                          "max_channel_width": 120, "min_channel_width": 10, "max_bank_height": 2, "river_corridor_width": 220, "min_bank_height": 0.75}}
}

In [43]:
def merge_dicts(dict_a: dict, dict_b: dict, replace_a: bool):
    """ Merge the contents of the dict_a and dict_b. Use recursion to merge
    any nested dictionaries. replace_a determines if the dict_a values are
    replaced or not if different values are in the dict_b.
    
    Adapted from https://stackoverflow.com/questions/7204805/how-to-merge-dictionaries-of-dictionaries

    Parameters:
            base_dict  The dict to 
            new_dict  The location of the centre of the river mouth
            replace_a If True any dict_a values are replaced if different values are in dict_b
    """
    def recursive_merge_dicts(base_dict: dict, new_dict: dict, replace_base: bool, path: list = []):
        """ Recurively add the new_dict into the base_dict. dict_a is mutable."""
        for key in new_dict:
            if key in base_dict:
                if isinstance(base_dict[key], dict) and isinstance(new_dict[key], dict):
                    recursive_merge_dicts(base_dict=base_dict[key], new_dict=new_dict[key], 
                                          replace_base=replace_base, path=path + [str(key)])
                elif base_dict[key] == new_dict[key]:
                    pass # same leaf value
                else:
                    if replace_base:
                        print(f"Conflict with both dictionaries containing different values at {path + [str(key)]}."
                              " Value replaced.")
                        base_dict[key] = new_dict[key]
                    else:
                        print(f"Conflict with both dictionaries containing different values at {path + [str(key)]}"
                              ". Value ignored.")
            else:
                base_dict[key] = new_dict[key]
        return base_dict
    
    return recursive_merge_dicts(copy.deepcopy(dict_a), dict_b, replace_base=replace_a)

In [44]:
shared = merge_dicts(dict_a=global_dict["shared"], dict_b=catchment_dict["shared"], replace_a=True)
instructions = {"rivers": shared, "waterways": shared, "dem": shared, "roughness": shared}

In [45]:
shared

{'data_paths': {'global_cache': '/nesi/project/niwa03440/cylc_test/'},
 'output': {'crs': {'horizontal': 2193, 'vertical': 7839}},
 'apis': {'vector': {'linz': {'land': {'layers': [51559]},
    'bathymetry_contours': {'layers': [50849]}}},
  'lidar': {'open_topography': {'NZ20_Westport': True}}},
 'general': {'bathymetry_contours_z_label': 'valdco',
  'lidar_classifications_to_keep': [2, 9],
  'elevation_range': [-10, 4000]}}

In [47]:
print('Add rivers')
instructions['rivers'] = merge_dicts(instructions['rivers'], global_dict['rivers'], replace_a=True)
instructions['rivers']['rivers']['network_file'] = instructions['rivers']['rivers'].pop("network_file_relative_path")
instructions['rivers'] = merge_dicts(instructions['rivers'], catchment_dict['rivers'], replace_a=True)
print('Add waterways')
instructions['waterways'] = merge_dicts(instructions['waterways'], global_dict['waterways'], replace_a=True)
print('Add dem')
instructions['dem'] = merge_dicts(instructions['dem'], global_dict['dem'], replace_a=True)
print('Add roughness')
instructions['roughness'] = merge_dicts(instructions['roughness'], global_dict['roughness'], replace_a=True)

Add rivers
Add waterways
Add dem
Add roughness
Conflict with both dictionaries containing different values at ['general', 'lidar_classifications_to_keep']. Value replaced.


In [48]:
instructions

{'rivers': {'data_paths': {'global_cache': '/nesi/project/niwa03440/cylc_test/'},
  'output': {'crs': {'horizontal': 2193, 'vertical': 7839},
   'grid_params': {'resolution': 1}},
  'apis': {'vector': {'linz': {'land': {'layers': [51559]},
     'bathymetry_contours': {'layers': [50849]}}},
   'lidar': {'open_topography': {'NZ20_Westport': True}}},
  'general': {'bathymetry_contours_z_label': 'valdco',
   'lidar_classifications_to_keep': [2, 9],
   'elevation_range': [-10, 4000],
   'drop_offshore_lidar': True,
   'interpolate_missing_values': False},
  'processing': {'chunk_size': 1400},
  'rivers': {'veg_lidar_classifications_to_keep': [2, 3, 4, 5, 9],
   'cross_section_spacing': 10,
   'network_columns': {'id': 'NZREACH',
    'to_node': 'to_node',
    'from_node': 'from_node',
    'flow': 'flow',
    'mannings_n': 'n',
    'area': 'CATCHAREA'},
   'network_file': 'data/rec1_flow_and_friction.geojson',
   'osm_id': 132793862,
   'area_threshold': 100000000,
   'minimum_slope': 0.002,


In [3]:
def merge(a, b, path=None):
    # From https://stackoverflow.com/questions/7204805/how-to-merge-dictionaries-of-dictionaries
    "merges b into a"
    if path is None: path = []
    for key in b:
        if key in a:
            if isinstance(a[key], dict) and isinstance(b[key], dict):
                merge(a[key], b[key], path + [str(key)])
            elif a[key] == b[key]:
                pass # same leaf value
            else:
                print(f"Conflict at {path + [str(key)]}")
                #raise Exception('Conflict at %s' % '.'.join(path + [str(key)]))""
        else:
            a[key] = b[key]
    return a

In [4]:
shared = merge(dict(global_dict["shared"]), catchment_dict["shared"])
instructions = {"rivers": shared, "waterways": shared, "dem": shared, "roughness": shared}
shared

{'data_paths': {'global_cache': '/nesi/project/niwa03440/cylc_test/'},
 'output': {'crs': {'horizontal': 2193, 'vertical': 7839}},
 'apis': {'vector': {'linz': {'land': {'layers': [51559]},
    'bathymetry_contours': {'layers': [50849]}}},
  'lidar': {'open_topography': {'NZ20_Westport': True}}},
 'general': {'bathymetry_contours_z_label': 'valdco',
  'lidar_classifications_to_keep': [2, 9],
  'elevation_range': [-10, 4000]}}

In [ ]:
instructions['rivers'] = merge(dict(instructions['rivers']), global_dict['rivers'])
instructions['rivers']['rivers']['network_file'] = instructions['rivers']['rivers'].pop("network_file_relative_path")
instructions['rivers'] = merge(dict(instructions['rivers']), catchment_dict['rivers'])

instructions['waterways'] = merge(dict(instructions['waterways']), global_dict['waterways'])

instructions['dem'] = merge(dict(instructions['dem']), global_dict['dem'])

instructions['roughness'] = merge(dict(instructions['roughness']), global_dict['roughness'])

In [5]:
global_dict['waterways']

{'output': {'grid_params': {'resolution': 1}},
 'processing': {'chunk_size': 2500},
 'drains': {'widths': {'drain': 5, 'stream': 7.5, 'river': 10}}}

In [9]:
instructions['waterways']

{'data_paths': {'global_cache': '/nesi/project/niwa03440/cylc_test/'},
 'output': {'crs': {'horizontal': 2193, 'vertical': 7839},
  'grid_params': {'resolution': 1}},
 'apis': {'vector': {'linz': {'land': {'layers': [51559]},
    'bathymetry_contours': {'layers': [50849]}}},
  'lidar': {'open_topography': {'NZ20_Westport': True}}},
 'general': {'bathymetry_contours_z_label': 'valdco',
  'lidar_classifications_to_keep': [2, 9],
  'elevation_range': [-10, 4000]}}

In [7]:
merge(dict(instructions['waterways']), global_dict['waterways'])

{'data_paths': {'global_cache': '/nesi/project/niwa03440/cylc_test/'},
 'output': {'crs': {'horizontal': 2193, 'vertical': 7839},
  'grid_params': {'resolution': 1}},
 'apis': {'vector': {'linz': {'land': {'layers': [51559]},
    'bathymetry_contours': {'layers': [50849]}}},
  'lidar': {'open_topography': {'NZ20_Westport': True}}},
 'general': {'bathymetry_contours_z_label': 'valdco',
  'lidar_classifications_to_keep': [2, 9],
  'elevation_range': [-10, 4000]},
 'processing': {'chunk_size': 2500},
 'drains': {'widths': {'drain': 5, 'stream': 7.5, 'river': 10}}}

In [10]:
merge(dict(instructions['roughness']), global_dict['roughness'])

Conflict at ['general', 'lidar_classifications_to_keep']


{'data_paths': {'global_cache': '/nesi/project/niwa03440/cylc_test/'},
 'output': {'crs': {'horizontal': 2193, 'vertical': 7839},
  'grid_params': {'resolution': 1}},
 'apis': {'vector': {'linz': {'land': {'layers': [51559]},
    'bathymetry_contours': {'layers': [50849]}}},
  'lidar': {'open_topography': {'NZ20_Westport': True}}},
 'general': {'bathymetry_contours_z_label': 'valdco',
  'lidar_classifications_to_keep': [2, 9],
  'elevation_range': [-10, 4000],
  'interpolation_method': 'linear'},
 'processing': {'chunk_size': 300}}